# Esercizio 4 - Segmentation

## Approccio

L'algoritmo è *brute force* perchè per trovare i tagli corretti li prova tutti generandoli in maniera randomica e associato ad ogni
sequenza di tagli un punteggio. Il punteggio è calcolato come la somma degli n termini più frequnti all'interno di ogni segmento.

L'idea alla base è che per ogni argomento ci siano un numero di parle che occorrono molte volte, e quindi l'algoritmo andrà a cercare 
i tagli che vanno a massimizzare questi valori, e che quindi vanno ad isolare i termini più frequenti per ogni segmento.

L'algoritmo ha bisogno del numero di tagli in ingresso.

- **COOCCORENCE** : In questo notebook corrispnde alla frequenza delle parole più utilizzate nel segmento

Il dataset è stato ripulito rimuovendo tutte le stopwords.

## Datasets

*data/segmentation_eng*: Il dataset è costituito da pezzi dei paragrafi di wikipedia di 4 argomenti diversi, in questo caso i 3 argomenti sono:
  - Gorillas
  - Quantum Computing
  - Astronomy
  - Kendrick Lamar Biography

I tagli corretti sono alla riga 59/60, alla riga 102/103 e alla riga 181/182.


### Imports

In [1]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random

### Methods

In [2]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

def get_text_from_file(path):
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def cooccorrence(text, n_most_words):
    '''
    Calculates the cooccorrence value of a sequence of words. 
    This value correspond to the sum of the occurrences of the n most frequently used words in the word list.
    '''
    score = 0
    c = Counter()
    most_common = []
    for row in text:
        c.update(row)
        
    most_common = c.most_common(n_most_words)
    # print(most_common)
    
    for el in most_common:
        score = score + el[1]
        
    return score

def extract_segment(file, start, end):
    '''
    Given the first and the last line, extract the segment.
    '''
    segment = []
    for i in range(start, end):
        segment.append(file[i])
    return segment

def get_cuts(ntopic):
    '''
    Generate a list of random value between 1 and the number of lines in the documents,
    that correspond to the cuts of the documents. The first value is 0 and the last is
    the numebers of lines in the documents.
    '''
    cut = [0] * ntopic
    while not(all(cut[i] < cut[i+1] for i in range(len(cut) - 1))):
        for k in range(1, ntopic): # generate the cuts
            cut[k] = random.randint(1, num_lines-1)
    cut.append(num_lines)
        
    return cut

def load_file(file_path):
    file = get_text_from_file(file_path)
    c = Counter()
    num_lines = sum(1 for line in open(file_path)) # Number of lines in the file

    for row in file:
        c.update(row)
        
    return num_lines, file, c
    

### Algorithm

In [3]:
def max_scores(file, ntopic, n_most_words, n_iteration): # n_most_words = number of most used words, used in cooccorrence()
    scores = [0]*ntopic
    sum_scores, max = 0, 0
    max_cut = []
    
    for f in range(n_iteration): 
        # First generate the random cuts
        cut = get_cuts(ntopic)
        
        # Extract the segments by the cuts and calculate the cooccorrence value for each segment
        # based on the random cuts
        for i in range(len(cut)-1):
            text = extract_segment(file, cut[i], cut[i+1])
            scores[i] = cooccorrence(text, n_most_words)
        
        # Evaluate the result and store the best result
        sum_scores = sum(scores)
        if(sum_scores > max):
            max = sum_scores
            max_cut = cut
            
    return max_cut, max

### Data Processing

In [8]:
n_most_words = 3
n_topics = 4
n_iteration = 20000 # Number of iteration to generate the best result - higher value = more time and better result
c = Counter()
file = '../data/segmentation_eng.txt'

file_data = load_file(file) # Load the file and get the number of lines and the list of most common word in the file

num_lines = file_data[0]
file = file_data[1]

for row in file:
    c.update(row)
    
print(c.most_common(30))

[('lamar', 78), ('quantum', 63), ('gorilla', 34), ('gorillas', 31), ('released', 30), ('also', 27), ('astronomy', 25), ('album', 25), ('first', 24), ('dre', 19), ('classical', 18), ('mixtape', 17), ('algorithm', 16), ('stars', 16), ('silverback', 15), ('early', 15), ('astronomical', 15), ('ray', 15), ('song', 15), ('years', 14), ('females', 14), ('made', 14), ('new', 14), ('computers', 14), ('computer', 14), ('wavelengths', 14), ('video', 14), ('males', 13), ('known', 13), ('may', 13)]


### File 1

In [9]:
res = max_scores(file, n_topics, n_most_words, n_iteration)

print(f'\nBest cut at lines -->  {res[0]}\nthe max sum is    -->  {res[1]}')



Best cut at lines -->  [0, 59, 105, 181, 254]
the max sum is    -->  363


### Analisi dei risultati e limiti di questo approccio

Per quanto semplice come approccio riesce a trovare i tegli corretti in poche esecuzioni sul nostro dataset, ma presenta diversi problemi:

- **scala male su testi grossi**: Purtroppo l'algoritmo per come è stato studiato scala male su dataset più grossi perchè deve esaminare
  più righe di testo e deve provare più combinazioni di taglie essendoci più righe ed eventualmente più tagli.

- **similarity**: Per valutare la similarity all'interno di un documento prendiamo la somma dei termini più frequenti all'interno di ogni segmento.
  Un possibile miglioramento è quello di trasformare il testo in un vettore e calcolare la cosine similarity del segmento e si va ad effettuare 
  il taglio dove la similarity va a decrescere per poi risalire, come abbiamo visto a lezione. Abbiamo deciso di non seguire questa strada per 
  trovare un approccio alternativo, anche se abbiamo ottenuto permormance peggiori sia in terimini di tempi di esecuzione che di precisione dei
  tagli.

- **dataset**: L'algoritmo performa bene su questo testo perchè in ogni segmento è presente una grande quantità di termini "unici", o meglio 
  che compaiono solamente all'interno di quel segmento. Se si tenta di applicare questo algoritmo a un dataset meno eterogeneo è probabile
  che i tagli siano più imprecisi.
